In [ ]:
pip install opencv-python-headless matplotlib numpy scikit-learn pytesseract gradio

In [ ]:
!sudo apt install tesseract-ocr tesseract-ocr-ben

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  tesseract-ocr-ben
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 516 kB of archives.
After this operation, 870 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-ben all 1:4.00~git30-7274cfa-1.1 [516 kB]
Fetched 516 kB in 0s (4,832 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting p

In [ ]:
import cv2
import numpy as np
from PIL import Image
import pytesseract
from google.colab.patches import cv2_imshow

# Load image
img = cv2.imread("/content/Licence.jpeg")  # use uploaded filename
if img is None:
    print("⚠️ Error: Image not found.")
    exit()

# Convert to grayscale and preprocess
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.equalizeHist(gray)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Edge detection and morphological closing
edges = cv2.Canny(blurred, 50, 150)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 7))
morph = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

# Find contours
contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter candidate regions
candidates = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    aspect_ratio = w / h
    if 2.0 < aspect_ratio < 6.0 and w > 100 and h > 40:
        candidates.append((x, y, w, h))

# Sort by area descending
candidates = sorted(candidates, key=lambda b: b[2]*b[3], reverse=True)

# Try OCR on candidates
detected_text = ""
for i, (x, y, w, h) in enumerate(candidates):
    plate_roi = img[y:y+h, x:x+w]
    roi_gray = cv2.cvtColor(plate_roi, cv2.COLOR_BGR2GRAY)
    _, roi_thresh = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Save & run OCR
    path = f"/content/plate_{i}.png"
    cv2.imwrite(path, roi_thresh)
    text = pytesseract.image_to_string(Image.open(path), lang='ben').strip()

    if len(text) > 4:
        detected_text = text
        # Show detected ROI with OCR success
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2_imshow(plate_roi)
        break

# Final result
if detected_text:
    print("✅ Detected License Plate Text:\n", detected_text)
    cv2_imshow(img)  # Show with bounding box
else:
    print("❌ No valid license plate detected.")


❌ No valid license plate detected.
